### Load imports and run the no LID case:

In [1]:
# Run this block to run the SWMM model on the 6-subcatchment Wingohocking model BEFORE adding GSI
%reset
import yaml
from swmm_objects import *
from swmm_read_cso import *
from swmm_run_cso import *
from subprocess import call
from pymongo import MongoClient
from datetime import datetime
import sys

#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
collection = 'july'
client = MongoClient(mongoServer)
swmmInpFile = "wingohocking_PWD.inp"  # initial SWMM .inp file
lidZeroFile = "PWD_wingo_Wakefield_list_zero.yaml"
################################################

db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database

thisRunCollection = db[collection]    # use or create the collection
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
f = open(lidZeroFile,'r')   # Read the no LID changable parameters from file
zeroLidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
zeroLidDocId = runswmm(zeroLidList,swmmInpStr,thisRunCollection)  # run SWMM on the no LID model for baseline
cursor = thisRunCollection.find_one({'_id':zeroLidDocId})
zeroLidPeak = cursor['peak']
zeroLidVolume = cursor['volume']
print "Stored record %s for zero LID usage:  peak = %s, volume = %s" % (zeroLidDocId,zeroLidPeak,zeroLidVolume)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
0.0 min, 12.21 sec
volume = 184.237
Stored record 5825e59344f6e30f30880641 for zero LID usage:  peak = 1906.92, volume = 184.237


### Loop ALL subcatchment LID numbers over a range:

In [2]:
# Now, apply a range of Wakefield Park rain gardens in all seven subcatchments simultaneously
from copy import deepcopy
for numberForAllLid in range(1,21):
    newRunList=deepcopy(zeroLidList)  # copy the zeroLidList into a new list
    for lid in newRunList:
        lid['Number'] = numberForAllLid  # the same number of LIDs will be in every subcatchment
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection)
    cursor = thisRunCollection.find_one({'_id':newRunDocId})
    newVolume = cursor['volume']
    volumeReduction = zeroLidVolume - newVolume
    print "Stored record %s for volume = %s" % (newRunDocId,newVolume)
    print "volumeReduction = %s" % (volumeReduction)
print "FINISHED ALL RUNS"

0.0 min, 13.48 sec
volume = 184.125
Stored record 5825e5a144f6e30f30880642 for volume = 184.125
volumeReduction = 0.112
0.0 min, 12.51 sec
volume = 184.012
Stored record 5825e5ae44f6e30f30880643 for volume = 184.012
volumeReduction = 0.225
0.0 min, 13.03 sec
volume = 183.901
Stored record 5825e5bb44f6e30f30880644 for volume = 183.901
volumeReduction = 0.336
0.0 min, 13.10 sec
volume = 183.789
Stored record 5825e5c944f6e30f30880645 for volume = 183.789
volumeReduction = 0.448
0.0 min, 12.42 sec
volume = 183.677
Stored record 5825e5d544f6e30f30880646 for volume = 183.677
volumeReduction = 0.56
0.0 min, 13.17 sec
volume = 183.564
Stored record 5825e5e344f6e30f30880647 for volume = 183.564
volumeReduction = 0.673
0.0 min, 12.53 sec
volume = 183.453
Stored record 5825e5ef44f6e30f30880648 for volume = 183.453
volumeReduction = 0.784
0.0 min, 12.28 sec
volume = 183.34
Stored record 5825e5fc44f6e30f30880649 for volume = 183.34
volumeReduction = 0.897
0.0 min, 12.15 sec
volume = 183.229
Stored 

### To display all of the runs from the database:

In [ ]:
runs = db[collection]
cursor = runs.find()
for run in cursor:
    print run
    print ''

{u'lidDict': None, u'swmmStartTime': u'2016-11-11 10:15:03.321000', u'evaporation': u'59.753', u'swmmInputFileStr': u'[TITLE]\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\nPolygons for the sheds: Projected Coordinate System:NAD_1983_StatePlane_Pennsylvania_South_FIPS_3702_Feet\n[OPTIONS]\nFLOW_UNITS\tCFS \nINFILTRATION\tGREEN_AMPT \nFLOW_ROUTING\tDYNWAVE \nLINK_OFFSETS\tDEPTH \nMIN_SLOPE\t0 \nALLOW_P

### To Reconstitute the SWMM input file for any of the runs above

In [ ]:
runs = db[collection]
cursor = runs.find_one({'_id':zeroLidDocId})
#print cursor
swmmInputFileStr = cursor["swmmInputFileStr"]
f = open("SWMM_reconstituted.inp",'w')
f.write(swmmInputFileStr)  # write out the swmmInputFileStr for modified problem
f.close()